In [207]:
import requests
import pandas as pd
import time
from dotenv import load_dotenv
import os

In [206]:
load_dotenv()

True

In [2]:
ev = pd.read_csv('ev-zipcode-demographics.csv')
ev.head()

,ZIP,Diesel,Electric,Flex_Fuel,Gasoline,Gasoline_Hybrid,Hydrogen,Natural_Gas,PHEV,Propane,...,Total_EV,EV_perc,Median_Household_Income,Latino_perc,White_perc,Asian_perc,Black_perc,BachOrHigher_perc,Total_Population,Zillow_Home_Value_Index
0,94027,89,723,137,4764,380,1,0,167,0,...,891,14.23,250001.0,8.8,68.0,18.5,1.2,84.7,7213,7397088.91
1,94022,228,1880,255,12682,1416,8,1,502,0,...,2390,14.08,250001.0,5.2,53.7,36.2,0.2,87.6,20069,4383734.10
2,94301,141,1229,163,8513,1141,5,5,352,0,...,1586,13.73,184932.0,6.2,59.6,24.4,2.5,82.0,17243,3763337.05
3,94028,128,645,110,4451,609,1,1,188,0,...,834,13.60,250001.0,9.0,79.5,5.9,0.0,85.2,6582,3708911.26
4,94024,253,1981,246,14374,1649,12,3,571,0,...,2564,13.43,250001.0,6.3,55.6,31.7,0.9,85.1,24203,3879869.05


In [194]:
ev.shape

(1427, 21)

In [ ]:
CATS = []
CATS.append('B02001') # race composition
CATS.append('B01003_001E') # total population
CATS.append('B19101C') # income
CATS.append('B19083_001E') # gini index (indicates income inequality)
CATS.append('B25003') # tenure
CATS

['B02001', 'B01003_001E', 'B19101C', 'B19083_001E', 'B25003']

In [210]:
# define query fields
API_KEY = os.environ.get('ACS_API_KEY')
YEAR = 2023
STATE_FIPS = "06"

for i in range(len(ev['ZIP'])):
    grid = []
    zipcode = str(ev['ZIP'].iloc[i])

    if (int(zipcode) in pd.read_csv('acs_data.csv')['zip code tabulation area'].to_list()):
        continue

    print(f'{i+1}/{len(ev['ZIP'])}:{zipcode}')
    for cat in CATS:
        CAT_NAME = cat if '_' in cat else f'group({cat})'
        # fetch data
        GEO_QUERY = f"zip%20code%20tabulation%20area:{zipcode}"
        url = f"https://api.census.gov/data/{YEAR}/acs/acs5?get=NAME,{CAT_NAME}&for={GEO_QUERY}&key={API_KEY}"
        data = requests.get(url).json()
        headers = data[0]
        result = data[1]
        population_data = dict(zip(headers, result))
        grid.append(population_data)
    row = pd.DataFrame(grid).groupby('zip code tabulation area').first().reset_index().dropna(axis=1, how='all')
    row.to_csv('acs_data.csv', mode='a', index=False, header=False)
    time.sleep(1)

In [98]:
variables_url = f"https://api.census.gov/data/{YEAR}/acs/acs5/variables"
variables = requests.get(variables_url).json()

In [99]:
keys = variables[0]
data_rows = variables[1:]

lookup = {
    row[0]: f"{row[2]}_{row[1]}"
    for row in data_rows
}

In [104]:
acs = pd.read_csv('acs_data.csv')
for col in acs.columns:
    try:
        if '_' in col:
            language_title = lookup[col]
            acs.rename({col: language_title}, axis=1, inplace=True)
    except:
        acs.drop(labels=[col], axis=1, inplace=True)

In [109]:
acs.rename({'zip code tabulation area': 'ZIP'}, axis=1, inplace=True)

In [110]:
acs.head()

,ZIP,NAME,Race_Estimate!!Total:,Race_Estimate!!Total:!!White alone,Race_Estimate!!Total:!!Black or African American alone,Race_Estimate!!Total:!!American Indian and Alaska Native alone,Race_Estimate!!Total:!!Asian alone,Race_Estimate!!Total:!!Native Hawaiian and Other Pacific Islander alone,Race_Estimate!!Total:!!Some Other Race alone,Race_Estimate!!Total:!!Two or More Races:,...,"Family Income in the Past 12 Months (in 2023 Inflation-Adjusted Dollars) (American Indian and Alaska Native Alone Householder)_Estimate!!Total:!!$60,000 to $74,999","Family Income in the Past 12 Months (in 2023 Inflation-Adjusted Dollars) (American Indian and Alaska Native Alone Householder)_Estimate!!Total:!!$75,000 to $99,999","Family Income in the Past 12 Months (in 2023 Inflation-Adjusted Dollars) (American Indian and Alaska Native Alone Householder)_Estimate!!Total:!!$100,000 to $124,999","Family Income in the Past 12 Months (in 2023 Inflation-Adjusted Dollars) (American Indian and Alaska Native Alone Householder)_Estimate!!Total:!!$125,000 to $149,999","Family Income in the Past 12 Months (in 2023 Inflation-Adjusted Dollars) (American Indian and Alaska Native Alone Householder)_Estimate!!Total:!!$150,000 to $199,999","Family Income in the Past 12 Months (in 2023 Inflation-Adjusted Dollars) (American Indian and Alaska Native Alone Householder)_Estimate!!Total:!!$200,000 or more",Gini Index of Income Inequality_Estimate!!Gini Index,Tenure_Estimate!!Total:,Tenure_Estimate!!Total:!!Owner occupied,Tenure_Estimate!!Total:!!Renter occupied
0,94027,ZCTA5 94027,7030,4909,57,4,1304,13,111,632,...,0,0,0,0,0,4,0.5178,2275,1989,286
1,94022,ZCTA5 94022,19993,10266,16,22,8165,51,270,1203,...,0,0,0,0,0,19,0.5045,7268,5862,1406
2,94301,ZCTA5 94301,17126,10136,377,2,4478,55,187,1891,...,0,0,0,0,0,0,0.5113,7080,3653,3427
3,94028,ZCTA5 94028,6645,5213,10,0,514,0,436,472,...,0,0,0,0,0,0,0.4950,2625,2220,405
4,94024,ZCTA5 94024,23673,12529,326,3,8373,85,479,1878,...,0,0,0,0,0,0,0.4765,8016,7063,953


In [113]:
fuel_counts = pd.read_csv('vehicle-fuel-type-counts-2024.csv')
fuel_counts.rename({'ZIP Code': 'ZIP'}, axis=1, inplace=True)
fuel_counts.head()

,Date,ZIP,Model Year,Fuel,Make,Duty,Vehicles
0,12/31/2024,90022,2012,Gasoline,HONDA,Light,280
1,12/31/2024,93263,2015,Gasoline,OTHER/UNK,Heavy,9
2,12/31/2024,93703,2014,Gasoline,HONDA,Light,83
3,12/31/2024,90640,<2011,Gasoline,HONDA,Light,1969
4,12/31/2024,96025,<2011,Gasoline,OTHER/UNK,Light,996


In [130]:
reform_fuel_counts = fuel_counts.groupby(['ZIP', 'Fuel'])['Vehicles'].sum().unstack(level='Fuel').fillna(0).reset_index()

In [133]:
reform_fuel_counts.head()

Fuel,ZIP,Battery Electric,Diesel and Diesel Hybrid,Flex-Fuel,Gasoline,Hybrid Gasoline,Hydrogen Fuel Cell,Natural Gas,Other,Plug-in Hybrid
0,90000,0.0,36.0,0.0,185.0,0.0,0.0,0.0,0.0,0.0
1,90001,309.0,763.0,1256.0,30161.0,1028.0,21.0,72.0,7.0,141.0
2,90002,294.0,500.0,1091.0,27272.0,962.0,14.0,6.0,3.0,102.0
3,90003,361.0,604.0,1370.0,36537.0,1312.0,22.0,22.0,1.0,146.0
4,90004,1691.0,239.0,473.0,25858.0,2545.0,22.0,7.0,0.0,550.0


In [134]:
ev.head()

,ZIP,Diesel,Electric,Flex_Fuel,Gasoline,Gasoline_Hybrid,Hydrogen,Natural_Gas,PHEV,Propane,...,Total_EV,EV_perc,Median_Household_Income,Latino_perc,White_perc,Asian_perc,Black_perc,BachOrHigher_perc,Total_Population,Zillow_Home_Value_Index
0,94027,89,723,137,4764,380,1,0,167,0,...,891,14.23,250001.0,8.8,68.0,18.5,1.2,84.7,7213,7397088.91
1,94022,228,1880,255,12682,1416,8,1,502,0,...,2390,14.08,250001.0,5.2,53.7,36.2,0.2,87.6,20069,4383734.10
2,94301,141,1229,163,8513,1141,5,5,352,0,...,1586,13.73,184932.0,6.2,59.6,24.4,2.5,82.0,17243,3763337.05
3,94028,128,645,110,4451,609,1,1,188,0,...,834,13.60,250001.0,9.0,79.5,5.9,0.0,85.2,6582,3708911.26
4,94024,253,1981,246,14374,1649,12,3,571,0,...,2564,13.43,250001.0,6.3,55.6,31.7,0.9,85.1,24203,3879869.05


In [138]:
ev_acs = pd.merge(ev, acs, on='ZIP',how='inner')

In [147]:
ev_zips = set(ev['ZIP'].unique())
acs_zips = set(acs['ZIP'].unique())
acs_only_zips = acs_zips - ev_zips
ev_only_zips = ev_zips - acs_zips

In [ ]:
ev_acs.to_csv('ev_acs.csv', index=False)

In [158]:
ev_acs.head()

,ZIP,Diesel,Electric,Flex_Fuel,Gasoline,Gasoline_Hybrid,Hydrogen,Natural_Gas,PHEV,Propane,...,"Family Income in the Past 12 Months (in 2023 Inflation-Adjusted Dollars) (American Indian and Alaska Native Alone Householder)_Estimate!!Total:!!$60,000 to $74,999","Family Income in the Past 12 Months (in 2023 Inflation-Adjusted Dollars) (American Indian and Alaska Native Alone Householder)_Estimate!!Total:!!$75,000 to $99,999","Family Income in the Past 12 Months (in 2023 Inflation-Adjusted Dollars) (American Indian and Alaska Native Alone Householder)_Estimate!!Total:!!$100,000 to $124,999","Family Income in the Past 12 Months (in 2023 Inflation-Adjusted Dollars) (American Indian and Alaska Native Alone Householder)_Estimate!!Total:!!$125,000 to $149,999","Family Income in the Past 12 Months (in 2023 Inflation-Adjusted Dollars) (American Indian and Alaska Native Alone Householder)_Estimate!!Total:!!$150,000 to $199,999","Family Income in the Past 12 Months (in 2023 Inflation-Adjusted Dollars) (American Indian and Alaska Native Alone Householder)_Estimate!!Total:!!$200,000 or more",Gini Index of Income Inequality_Estimate!!Gini Index,Tenure_Estimate!!Total:,Tenure_Estimate!!Total:!!Owner occupied,Tenure_Estimate!!Total:!!Renter occupied
0,94027,89,723,137,4764,380,1,0,167,0,...,0,0,0,0,0,4,0.5178,2275,1989,286
1,94022,228,1880,255,12682,1416,8,1,502,0,...,0,0,0,0,0,19,0.5045,7268,5862,1406
2,94301,141,1229,163,8513,1141,5,5,352,0,...,0,0,0,0,0,0,0.5113,7080,3653,3427
3,94028,128,645,110,4451,609,1,1,188,0,...,0,0,0,0,0,0,0.4950,2625,2220,405
4,94024,253,1981,246,14374,1649,12,3,571,0,...,0,0,0,0,0,0,0.4765,8016,7063,953


In [171]:
cal = pd.read_excel('calenviroscreen40resultsdatadictionary_F_2021.xlsx')
cal.head()

,Census Tract,Total Population,California County,ZIP,Approximate Location,Longitude,Latitude,CES 4.0 Score,CES 4.0 Percentile,CES 4.0 Percentile Range,...,Linguistic Isolation Pctl,Poverty,Poverty Pctl,Unemployment,Unemployment Pctl,Housing Burden,Housing Burden Pctl,Pop. Char.,Pop. Char. Score,Pop. Char. Pctl
0,6019001100,2780,Fresno,93706,Fresno,-119.781696,36.709695,93.183570,100.000000,95-100% (highest scores),...,79.374746,76.0,98.919598,12.8,93.831338,30.3,91.039290,93.155109,9.663213,99.722642
1,6077000700,4680,San Joaquin,95206,Stockton,-121.287873,37.943173,86.653790,99.987393,95-100% (highest scores),...,95.533902,73.2,98.391960,19.8,99.206143,31.2,92.281369,93.165408,9.664281,99.735250
2,6037204920,2751,Los Angeles,90023,Los Angeles,-118.197497,34.017500,82.393909,99.974786,95-100% (highest scores),...,81.553661,62.6,93.391960,6.4,61.530453,20.3,63.967047,83.751814,8.687785,95.789208
3,6019000700,3664,Fresno,93706,Fresno,-119.827707,36.734535,81.327940,99.962179,95-100% (highest scores),...,78.711598,65.7,95.351759,15.7,97.345133,35.4,96.413181,94.641227,9.817371,99.886536
4,6019000200,2689,Fresno,93706,Fresno,-119.805504,36.735491,80.745476,99.949571,95-100% (highest scores),...,86.561104,72.7,98.304020,13.7,95.288912,32.7,94.157161,95.398873,9.895964,99.949571


In [188]:
for idx, row in ev_acs.iterrows():
    zip = int(row.ZIP)
    x = cal.loc[cal['ZIP'] == zip, 'Housing Burden'].mean()
    ev_acs.loc[ev_acs['ZIP'] == zip, 'Housing Burden'] = x

In [ ]:
ev_acs.to_csv('ev_acs_with_calenviro.csv', index=False)